## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='SSSBSBBB'
meas='TOBS'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
1199
+-------------------+------------------+-------------------+---------+--------+--------+---------+-----------+------------------+-------------------+-------------------+-------------------+-----------+-----------+------+--------------------+------+
|            coeff_1|           coeff_2|            coeff_3|elevation|   label|latitude|longitude|measurement|             res_1|              res_2|              res_3|           res_mean|    station|  total_var|undefs|              vector|  year|
+-------------------+------------------+-------------------+---------+--------+--------+---------+-----------+------------------+-------------------+-------------------+-------------------+-----------+-----------+------+--------------------+------+
|-1053.3688766825214|-306.8152655949631|-14.795099503366163|   1813.6|SSSBSBBB| 39.3833|-117.3167|       TOBS|0.4788259752532819|0.43347041481144694|0.43355307657552833|0.1993353

In [5]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [6]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_1, coeff_2, coeff_3'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_1, coeff_2, coeff_3 FROM weather


,station,year,coeff_1,coeff_2,coeff_3
0,USC00261630,1974.0,-1053.368877,-306.815266,-14.795100
1,USC00042338,1989.0,-1011.871666,521.517027,-141.101874
2,USC00048331,1948.0,911.962999,-94.836559,300.822356
3,USC00044713,1960.0,-1053.962337,-31.063959,74.061414
4,USC00047085,1979.0,-924.051834,-13.752208,130.712405


In [7]:
year_station_table_1=pdf.pivot(index='year', columns='station', values='coeff_1')
year_station_table_2=pdf.pivot(index='year', columns='station', values='coeff_2')
year_station_table_3=pdf.pivot(index='year', columns='station', values='coeff_3')
# year_station_table_4=pdf.pivot(index='year', columns='station', values='coeff_4')
year_station_table_1.head(10)

station,USC00040931,USC00041018,USC00041462,USC00041912,USC00042334,USC00042338,USC00042467,USC00042500,USC00044288,USC00044713,...,USS0015K03S,USS0017K04S,USS0019K04S,USS0019K07S,USS0019K08S,USS0020K05S,USS0020K13S,USS0020K27S,USS0020K30S,USS0020K31S
year,,,,,,,,,,,,,,,,,,,,,
1901.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1902.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1903.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1904.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1905.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-957.133177,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1906.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1204.656309,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1907.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1908.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-877.740074,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1909.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [11]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

def MS(Mat):
    return np.nanmean(Mat**2)

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

total RMS                   =  652.312384947
RMS removing mean-by-station=  641.899529734
RMS removing mean-by-year   =  281.698896841


In [8]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

def MS(Mat):
    return np.nanmean(Mat**2)

In [9]:
mean_by_year_1=np.nanmean(year_station_table_1,axis=1)
mean_by_station_1=np.nanmean(year_station_table_1,axis=0)
tbl_minus_year_1 = (year_station_table_1.transpose()-mean_by_year_1).transpose()
tbl_minus_station_1 = year_station_table_1-mean_by_station_1
print 'coeff1 : '
print 'total MS                   = ',MS(year_station_table_1)
print 'MS removing mean-by-station= ',MS(tbl_minus_station_1),', fraction explained = ',100*(MS(year_station_table_1) - MS(tbl_minus_station_1))/MS(year_station_table_1)
print 'MS removing mean-by-year   = ',MS(tbl_minus_year_1),', fraction explained = ',100*(MS(year_station_table_1) - MS(tbl_minus_year_1))/MS(year_station_table_1)
print ''

mean_by_year_2=np.nanmean(year_station_table_2,axis=1)
mean_by_station_2=np.nanmean(year_station_table_2,axis=0)
tbl_minus_year_2 = (year_station_table_2.transpose()-mean_by_year_2).transpose()
tbl_minus_station_2 = year_station_table_2-mean_by_station_2

print 'total MS                   = ',MS(year_station_table_2)
print 'MS removing mean-by-station= ',MS(tbl_minus_station_2),', fraction explained = ',100*(MS(year_station_table_2) - MS(tbl_minus_station_2))/MS(year_station_table_2)
print 'MS removing mean-by-year   = ',MS(tbl_minus_year_2),', fraction explained = ',100*(MS(year_station_table_2) - MS(tbl_minus_year_2))/MS(year_station_table_2)
print ''

mean_by_year_3=np.nanmean(year_station_table_3,axis=1)
mean_by_station_3=np.nanmean(year_station_table_3,axis=0)
tbl_minus_year_3 = (year_station_table_3.transpose()-mean_by_year_3).transpose()
tbl_minus_station_3 = year_station_table_3-mean_by_station_3

print 'total MS                   = ',MS(year_station_table_3)
print 'MS removing mean-by-station= ',MS(tbl_minus_station_3),', fraction explained = ',100*(MS(year_station_table_3) - MS(tbl_minus_station_3))/MS(year_station_table_3)
print 'MS removing mean-by-year   = ',MS(tbl_minus_year_3),', fraction explained = ',100*(MS(year_station_table_3) - MS(tbl_minus_year_3))/MS(year_station_table_3)
print ''

# mean_by_year_4=np.nanmean(year_station_table_4,axis=1)
# mean_by_station_4=np.nanmean(year_station_table_4,axis=0)
# tbl_minus_year_4 = (year_station_table_4.transpose()-mean_by_year_4).transpose()
# tbl_minus_station_4 = year_station_table_4-mean_by_station_4

# print 'total MS                   = ',MS(year_station_table_4)
# print 'MS removing mean-by-station= ',MS(tbl_minus_station_4),', fraction explained = ',100*(MS(year_station_table_4) - MS(tbl_minus_station_4))/MS(year_station_table_4)
# print 'MS removing mean-by-year   = ',MS(tbl_minus_year_4),', fraction explained = ',100*(MS(year_station_table_4) - MS(tbl_minus_year_4))/MS(year_station_table_4)
# print ''

coeff1 : 
total MS                   =  780877.35849
MS removing mean-by-station=  224304.309716 , fraction explained =  71.2753472389
MS removing mean-by-year   =  614226.266642 , fraction explained =  21.3415192587

total MS                   =  62083.8344165
MS removing mean-by-station=  26752.6897642 , fraction explained =  56.908766967
MS removing mean-by-year   =  44329.5641975 , fraction explained =  28.5972514195

total MS                   =  43538.0148575
MS removing mean-by-station=  33806.525223 , fraction explained =  22.3517072755
MS removing mean-by-year   =  25288.7357648 , fraction explained =  41.9157353693



In [12]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 652.312384947
0 after removing mean by year    = 281.698896841
0 after removing mean by stations= 268.442877822
1 after removing mean by year    = 267.084176624
1 after removing mean by stations= 266.678831215
2 after removing mean by year    = 266.510870048
2 after removing mean by stations= 266.431313173
3 after removing mean by year    = 266.390236857
3 after removing mean by stations= 266.367238732
4 after removing mean by year    = 266.353175433
4 after removing mean by stations= 266.343730149
